# Animation index

    // File header
    // 0x00000000: DGD<16:nAnimations>
    // 0x0000001B: < 2:HEADER_SIZE>	//??? HEADER_SIZE = 0x200 !
    // 0x000001EF: < 3:"B130"> 		//build string
    // startup animation
    // 0x00001400: < 7:"RUN_DMD"> 	//marker of begin of retentive settings area?
    // 0x0000C600:					//Unused OS bootsector data
    // 0x0000C800:					//Begin of animation table (see below)
    // 0x000E8000:					//Looks like a crapload of font data!
    // 0x004E2000:					//First animation frame

    // 0x00002710:					//First raw offset
    // --> MAGIC_OFFSET = 0

    Starting from 0xC800, there seems to be a 0x200 size header block for each animation

    ---------------------------------------------------------------------
    0001 02 03 04050607 08 09 0A 0B 0C 0D 0E 0F 10 11 12 13 <string name> <0x00 padding>
    ---------------------------------------------------------------------
    0001 01 58 000061A8 70 57 20 02 01 01 01 2E 03 2F 3B 01 24_001              
    0002 01 2C 00006309 38 57 20 02 01 01 01 5A 0A 01 2C 01 24_002              
    0003 01 06 000063BA 18 29 20 01 00 00 00 00 00 00 00 00 24_003             
    0004 01 0A 000063D3 1E 22 20 01 00 00 00 00 00 00 00 00 24_004             
    0005 01 05 000063FC 1E 83 20 01 00 00 00 00 00 00 00 01 24_005             
    0006 01 29 00006411 29 57 20 02 01 01 01 05 0A 01 29 01 24_006             
    0007 01 08 000064B6 30 AE 20 01 00 00 00 00 00 00 00 01 24_007              
    0008 01 1D 000064D7 1D 17 1D 02 01 01 01 03 0A 01 23 01 24_008              
    0009 01 3F 0000654C 50 57 20 02 01 01 01 03 0A 01 3F 00 24_009              
    000A 01 10 00006649 10 80 20 01 01 01 00 00 00 00 00 00 AC#DC_001           
    000B 01 20 0000668A 20 80 20 01 01 00 00 00 00 00 00 00 AC#DC_002
    022E 01 01 0000FFFD 01 80 20 01 00 00 00 00 00 00 00 00 FLINTSTONES_024
    022F 01 01 00010002 01 80 20 01 00 00 00 00 00 00 00 00 FLINTSTONES_025
    0230 01 01 00010007 01 80 20 01 00 00 00 00 00 00 00 00 FLINTSTONES_026
    08FC 01 23 00034AB7 23 80 20 01 01 00 00 00 00 00 00 00 X-MEN_063
    08FD 01 14 00034B44 14 80 20 01 00 00 00 00 00 00 00 00 X-MEN_064
    08FE 01 39 00034B95 39 80 20 00 01 01 00 00 00 00 00 00 X-MEN_065
    
    * Watch out, header has been clipped off

__Confirmed__

        0001 : Animation ID
          02 : ???
          03 : Number of stored frames
    04050607 : get byteOffset to frameTable:    x  * 0x200
                              firstFrame at: (1+x) * 0x200
          08 : number of frame entries in the frameTable
          09 : xResolution [pixels] (sometimes has errors)
          0A : yResolution [pixels] (sometimes has errors)


## Frame table (animation header)

    N * 2 bytes: 
      <frame number to show> <miliseconds to show frame>
    the frame number can be 0 sometimes, show a blank frame then
    the miliseconds is never zero, which is used to confirm a valid table
    Actual frame data starts at frameTable + 0x200   


In [ ]:
offset = 0x049A1000
fOffs=0
headerDat = rawDat[offset:offset+0x200]
getHeader( headerDat )[0]

In [ ]:
d.set_data( getFrame(rawDat, byteOffset=offset+0x200, frameOffset=fOffs) )
print(fOffs)
fOffs += 1

In [ ]:
255/15

In [ ]:
15*17

In [ ]:
d.set_data( getFrame(rawDat, byteOffset=offset+0x200, frameOffset=fOffs) )
print(fOffs)
fOffs += 1

In [ ]:
%pylab
import time
import matplotlib.animation as animation

def hexDump( buffer ):
    """ Print a pretty hex-dump on the debug out """
    for i,b in enumerate(buffer):
        if len(buffer)>16 and (i%16)==0:
            print( "\n    {:04x}: ".format(i), end='' )
        print( "{:02x} ".format(b), end='' )

def getHeader( buf ):
    """ return frame index, frame duration [ms] """
    frameId = buf[0::2]
    frameMs = buf[1::2]
    return frameId, frameMs        
        
def getFrame( buf, xSize=128, ySize=32, frameOffset=0, byteOffset=0 ):
    """ returns unpacked image data, 1 byte per pixel """
    byteOffset += xSize*ySize*frameOffset//2
    rawDat = buf[byteOffset:byteOffset+ySize*xSize//2]
    # unpack 2 pixels / byte to 1 pixel / byte
    a = zeros(xSize*ySize, dtype=uint8)
    a[1::2] = rawDat & 0x0F
    a[0::2] = rawDat >> 4
    return a.reshape((ySize,xSize))

def isHeader( dat ):
    """
    returns offset to the beginning of a detected header frame.
    returns -1 of no header was detected
    """
    tag = b"\x01\x02\x03\x04\x05"
    a = b"".join( dat.flatten()[0::2] )
    b = b"".join( dat.flatten()[1::2] )
    ind = a.find( tag )
    if ind >= 0:
        return 2*ind
    ind = b.find( tag )
    if ind >= 0:
        return 2*ind+1
    return ind

fNameOffset = 0xC814
def getNextFname():
    global fNameOffset
    fName = b"".join( rawDat[fNameOffset:fNameOffset+32] ).replace(b"\x00",b"").decode("ascii")
    fNameOffset += 0x200
    return fName

fName = "./runDmd.img"
#fName = "./RunDMD_B130.img"
#fName = "/home/michael/Desktop/older.img"
#fName = "/media/sf_Google_Drive/pdmd/picodmd_b236.img"
with open(fName,"rb") as f:
    rawDat = array( bytearray(f.read()), dtype=uint8 )

# Parse header

In [ ]:
headerStartOffset = 0xC800
hexDump( rawDat[headerStartOffset:headerStartOffset+100] )

# Animated plot

In [ ]:
fNameOffset = 0
xSize = 128
ySize = 32  #len(rawDat)
frameSize = xSize * ySize // 2
offset = 0x0698F000

close("all")
figure( figsize=(8,2))
d = imshow( getFrame(rawDat, xSize, ySize, 0, offset), aspect="equal", vmin=0, vmax=15, interpolation="hanning", cmap=cm.binary_r )

def ani(x):
    global offset
    aniDat = rawDat[offset:offset+frameSize]
    fOffs = isFooter( aniDat )
    if fOffs >= 0:
        offset += fOffs + 0x200
        return
    d.set_data( getFrame(aniDat, xSize, ySize) )
    offset += frameSize

#ani = animation.FuncAnimation(gcf(), ani, interval=50 )

In [ ]:
from PIL import Image

# Gif write test

In [ ]:
offset = 0x12C400
imgs = []
for i in range(5):
    img = Image.fromarray( getFrame(rawDat, byteOffset=offset+frameSize*i)*16 )
    img.info["transparency"] = 0x0A*16
    img.info["duration"] = 1000
    imgs.append( img )
imgs[0].save("test.gif", "GIF", optimize=False, save_all=True, append_images=imgs[1:] )

# Gif export

In [ ]:
xSize = 128
ySize =  32
frameSize = xSize * ySize // 2
headerSize = 0x200
offset = 0x0013EC00
fNameOffset = 0xC814
cnter = 0
while(1):  # for each animation
    imlist = []
    # find new header
    #headerDat = rawDat[offset:offset+2**17]
    #headerOffset = isHeader( headerDat )
    #if headerOffset < 0:
    #    print("Header not found :(")
    #    break
    #offset += headerOffset
    # decode header
    headerDat = rawDat[offset:offset+headerSize]
    offset += headerSize
    # decode frames
    fName = getNextFname()
    headDat = getHeader( headerDat )
    for frameId, frameDel in zip( *headDat ):
        if frameId<=0 or frameDel<=0:
            continue
        #print( frameId, frameDel, "  ", end="" )
        imgData = getFrame( rawDat, xSize, ySize, byteOffset=offset, frameOffset=frameId-1 ) * 16
        img = Image.fromarray(imgData, mode="L")
        img.info["duration"] = frameDel
        imlist.append( img )
    print( hex(offset), len(imlist), fName )
    #print("")
    # end of current file, save it
    fullFName = "./gif/{}.gif".format( fName )
    imlist[0].save( fullFName, "GIF", optimize=True, save_all=True, transparency=0x0A*16, append_images=imlist[1:] )
    offset += amax( headDat[0] ) * frameSize
    #if cnter > 50:
    #    break
    #cnter += 1

In [ ]:
hex(0x2b0a00 + frameSize*19)

In [ ]:
isFooter( rawDat[0x383400:] )

In [ ]:
ftr = getFrame(rawDat, byteOffset=0x1A2A00 + frameSize*0 )
frm = getFrame(rawDat, byteOffset=0x1A2A00 + frameSize*-1 )

In [ ]:
isFooter( ftr )

In [ ]:
i=0
startOffset = 0x188600

In [ ]:
frm = getFrame(rawDat, byteOffset=startOffset+frameSize*i)
d.set_data( frm )
print(hex(i))
print(hex(startOffset+frameSize*i))
i += 1

# Backport from C

In [ ]:
%pylab
from struct import pack, unpack
HEADER_OFFS   = 0x0000C800
HEADER_SIZE   = 0x00000200
ANI_DAT_START = 0x004E2000

In [ ]:
f = open("./runDmd.img", "rb")
if f.read(3) != b'DGD':
    raise RuntimeError('Invalid DGD header')
nAnimations = unpack(">H", f.read(2))

In [ ]:
nAnimations

In [ ]:
f.seek(0x1ef)
buf = f.read(8)
buildStr = buf.strip(b'\x00').decode("ascii")

In [ ]:
buildStr

In [ ]:
def parse_header(f, headerIndex):
    """
      f: file object of RunDMD_BXXX.img
      headerIndex: integer, which animation to fetch
      returns: dict with header info
    """
    f.seek(HEADER_OFFS + HEADER_SIZE * headerIndex)
    rawDat = unpack(">HxBIBBB9x32s", f.read(52))
    header = {
        "animation_id": rawDat[0],
        "n_bitmaps": rawDat[1],
        "byte_offset": rawDat[2] * HEADER_SIZE,
        "n_frames": rawDat[3],
        "width": rawDat[4],
        "height": rawDat[5],
        "name": rawDat[6].strip(b'\x00').decode("ascii")
    }
    f.seek(header["byte_offset"])
    buff = f.read(header["n_frames"] * 2)
    header["frame_ids"] = list(buff[0::2])
    header["frame_durs"] = list(buff[1::2])
    return header

def get_frame_dat(f, header, frame_offset):
    """
        f: open .img file
        header: dict from parse_header()
        frame_offset: which frame (0 = first)
        returns byte buffer of size 128 x 64, 1 pixel per byte
    """
    byte_offset = header["byte_offset"] + HEADER_SIZE
    byte_offset += 128 * 32 * frame_offset // 2
    f.seek(byte_offset)
    buff_4 = f.read(128 * 32 // 2)      # 2 pixel per byte
    # unpack 2 pixels / byte to 1 pixel / byte
    buff_8 = bytearray(128 * 32)        # 1 pixel per byte
    buff_8[0::2] = [(i & 0xF0) for i in buff_4]
    buff_8[1::2] = [(i & 0x0F) << 4 for i in buff_4]
    return bytes(buff_8)

In [ ]:
h = parseHeader(f, 900)
h

In [ ]:
dat = get_frame_dat(f, h, 0)
Image.frombytes("L", (128, 32), dat)

In [ ]:
h = parse_header(f, 1)
imgs = []
for f_id, f_dur in zip(h["frame_ids"], h["frame_durs"]):
    img_dat = get_frame_dat(f, h, f_id - 1)
    img = Image.frombytes("L", (128, 32), img_dat)
    img.info["background"] = 0x00 << 4
    img.info["transparency"] = 0x0A << 4
    img.info["duration"] = f_dur
    imgs.append(img)